In [43]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime, timedelta

# 設置MySQL資料庫連接
db_user = 'root'
db_password = '19970730'
db_host = '127.0.0.1'
db_name = 'sql_stock'

# 創建資料庫連接引擎
engine = create_engine(f'mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}')

# 設置日期範圍
#today = datetime(2024, 5, 25)
today = datetime.now().date()
one_year_ago = today - timedelta(days=1000)

# 格式化日期
today_str = today.strftime('%Y%m%d')
one_year_ago_str = one_year_ago.strftime('%Y%m%d')

# 可自由調整select的區間
today_str = '20240807'
one_year_ago_str = '20210101'

# SQL 查詢 加權指數
query_twse = f"""
SELECT *
FROM daily_twse
WHERE date >= '{one_year_ago_str}' AND date <= '{today_str}'
"""

# SQL 查詢 個股
query_stock = f"""
SELECT *
FROM daily_price
WHERE date >= '{one_year_ago_str}' AND date <= '{today_str}'
"""

# SQL 查詢 細產業
query_sub_category = f"""
SELECT *
FROM sub_category
"""

# 使用 pd.read_sql 來執行查詢並讀取數據到 DataFrame
twse_df_raw = pd.read_sql(query_twse, engine)

stock_df_raw = pd.read_sql(query_stock, engine)

sub_category_list = pd.read_sql(query_sub_category, engine)



In [44]:
import pandas as pd
import pymysql

class MySQLUpdater:
    def __init__(self, db_params, table_name, df_stocks, condition, method):
        self.db = pymysql.connect(**db_params)
        self.cursor = self.db.cursor()
        self.table_name = table_name
        self.df_stocks = df_stocks
        self.condition = condition
        self.method = method
        


    def update_records(self):
        # 遍历 DataFrame 中的每一行

        for i, row in self.df_stocks.iterrows():
            try:
                if(self.method == '1'):
                    # 构建 SQL 更新语句
                    sql = f"""
                    UPDATE `{self.table_name}`
                    SET `240MA` = %s,
                        `240RS` = %s
                    WHERE `{self.condition}` = %s AND `Date` = %s
                    """
                    # 执行更新语句
                    self.cursor.execute(sql, (float(row['240MA']),float(row['240RS']), row[self.condition], row['Date']))

                    # 提交更改
                    self.db.commit()
                    
                elif (self.method == '2'):

                    # 构建 SQL 更新语句 更新標準化RS
                    sql = f"""
                    UPDATE `{self.table_name}`
                    SET 
                        `240RS_sd` = %s,
                        `240RS_rank` = %s
                    WHERE `{self.condition}` = %s AND `Date` = %s
                    """

                    # 执行更新语句
                    self.cursor.execute(sql, (float(row['240RS_sd']),float(row['240RS_rank']), row[self.condition], row['Date']))

                    # 提交更改
                    self.db.commit()

                # elif (self.method == '3'):

                #     # 构建 SQL 更新语句 更新標準化RS
                #     sql = f"""
                #     UPDATE `{self.table_name}`
                #     SET `60RS_sd` = %s,
                #         `60RS_rank` = %s
                #     WHERE `{self.condition}` = %s AND `Date` = %s
                #     """

                #     # 执行更新语句
                #     self.cursor.execute(sql, (float(row['60RS_sd']),float(row['60RS_rank']), row[self.condition], row['Date']))

                #     # 提交更改
                #     self.db.commit()

            except Exception as e:
                print(f"Error updating row {i}: {e}")
                self.db.rollback()

    def close_connection(self):
        self.cursor.close()
        self.db.close()

# 示例使用
db_params = {
    'host': '127.0.0.1',
    'user': 'root',
    'password': '19970730',
    'database': 'sql_stock'
    # 設置MySQL資料庫連接
}

In [14]:
# twse_df = twse_df_raw[twse_df_raw['指數名稱'] == '發行量加權股價指數'].copy()

# twse_df_na = twse_df.isna()
# twse_df_update = twse_df[twse_df_na.any(axis=1)] #需要被更新的欄位

# twse_df.loc[:, '240MA']= twse_df['價格指數值'].rolling(240).mean()

# twse_df.loc[:, '240RS'] = round((twse_df['價格指數值']/twse_df['240MA'])*100, 4)
# twse_df = twse_df.fillna(0)
# #twse_df_new = twse_df.iloc[-2:]
# twse_df_new = twse_df.loc[twse_df_update.index]  # 只更新尚未計算的新欄位


# # 初始化并更新数据库
# updater = MySQLUpdater(db_params, 'daily_twse', twse_df_new, '指數名稱', '1')
# updater.update_records()
# updater.close_connection()

In [45]:
import numpy as np
stock_ids = stock_df_raw['證券代號'].unique().tolist()

stock_df_na = stock_df_raw.isna()
stock_df_update = stock_df_raw[stock_df_na.any(axis=1)] #需要被更新的欄位

for i in stock_ids:
        if((len(i)==4) ): #1. 排除債 ETF等等 2.排除新掛牌資料不到calculate_day天
                #print(i)
                # 計算個股 RS
                temp_df = stock_df_raw[stock_df_raw['證券代號']==i].copy()

                temp_df.loc[:, '240MA']= temp_df['收盤價'].rolling(240).mean()

                # 避免 temp_df['20MA']為0 
                if (temp_df['收盤價'] == 0).any():
                        temp_df.loc[:, '240RS'] = 0
                else :
                        temp_df.loc[:, '240RS'] = round((temp_df['收盤價']/temp_df['240MA'])*100, 4)


                temp_df = temp_df.fillna(0)

                #只更新尚未update欄位
                temp_update_df = stock_df_update[stock_df_update['證券代號']==i].copy()
                temp_df_new = temp_df.loc[temp_update_df.index]  # 只更新尚未計算的新欄位
                # 初始化并更新数据库
                updater = MySQLUpdater(db_params, 'daily_price', temp_df_new, '證券代號','1')
                updater.update_records()
updater.close_connection()

In [46]:
# 

import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime, timedelta

# 設置MySQL資料庫連接
db_user = 'root'
db_password = '19970730'
db_host = '127.0.0.1'
db_name = 'sql_stock'

# 創建資料庫連接引擎
engine = create_engine(f'mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}')

# 設置日期範圍
#today = datetime(2024, 5, 25)
start_date = pd.to_datetime(stock_df_update['Date']).unique().tolist()[0].date()
end_date = pd.to_datetime(stock_df_update['Date']).unique().tolist()[-1].date()

# 格式化日期
start_date = start_date.strftime('%Y%m%d')
end_date = end_date.strftime('%Y%m%d')

# SQL 查詢 個股
query_stock = f"""
SELECT *
FROM daily_price
WHERE date >= '{start_date}' AND date <= '{end_date}'
"""

stock_df_update_done = pd.read_sql(query_stock, engine)


In [47]:
twse_df_raw[twse_df_raw['指數名稱'] == '發行量加權股價指數'].copy()

,Date,指數名稱,價格指數值,報酬指數值,漲跌點數,漲跌百分比,5MA,8MA,10MA,20MA,20RS,60MA,60RS,240MA,240RS
22,2021-01-04,發行量加權股價指數,14902.03,+,169.50,1.15,0.000,0.0000,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
74,2021-01-05,發行量加權股價指數,15000.03,+,98.00,0.66,0.000,0.0000,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
126,2021-01-06,發行量加權股價指數,14983.13,-,16.90,-0.11,0.000,0.0000,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
178,2021-01-07,發行量加權股價指數,15214.00,+,230.87,1.54,0.000,0.0000,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
230,2021-01-08,發行量加權股價指數,15463.95,+,249.95,1.64,15112.628,0.0000,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46164,2024-08-01,發行量加權股價指數,22642.10,+,442.75,1.99,22269.744,22418.3513,22651.510,23236.7765,97.4408,22341.3623,101.3461,18976.8402,119.3144
46220,2024-08-02,發行量加權股價指數,21638.09,-,1004.01,-4.43,22173.520,22264.4550,22438.337,23160.0595,93.4285,22357.7717,96.7811,18996.6778,113.9046
46276,2024-08-05,發行量加權股價指數,19830.88,-,1807.21,-8.35,21706.798,21961.1913,22081.578,22975.4770,86.3132,22343.2778,88.7555,19009.0109,104.3236
46332,2024-08-06,發行量加權股價指數,20501.02,+,670.14,3.38,21362.288,21664.8388,21844.754,22822.6985,89.8273,22342.2820,91.7588,19025.1206,107.7576


In [48]:
# 計算個股對大盤RS
#先轉置取Date
stock_df_update_done['Date'] = pd.to_datetime(stock_df_update_done['Date'])

#stock_date_list = stock_df_raw['Date'].unique().tolist()
stock_date_list =  pd.to_datetime(stock_df_update['Date']).unique().tolist()  # 指查詢要更新的日期

twse_df = twse_df_raw[twse_df_raw['指數名稱'] == '發行量加權股價指數'].copy()
twse_df['Date'] = pd.to_datetime(twse_df['Date'])  # 要注意一下是不是 twse已經有資料，這樣twse_df_new會空掉就沒辦法更新RS 因為是跟大盤在比
twse_df.rename(columns={'240RS': '240RS_twse'}, inplace=True)
cols = ['Date', '價格指數值', '240RS_twse']
twse_df_forRS = twse_df[cols]

merged_df = pd.merge(stock_df_update_done, twse_df_forRS, on='Date', how='inner')
merged_df['240RS_sd'] = merged_df['240RS']/merged_df['240RS_twse']  #240MA計算RS
merged_df = merged_df.fillna(0)

col2 = ['Date', '證券代號','240RS_sd','240RS_rank']
for i in range(len(stock_date_list)) :
    temp_merged_df = merged_df[merged_df['Date']==stock_date_list[i]].copy()
    # #標準化成0-100名
    temp_merged_df['240RS_rank'] = (temp_merged_df['240RS_sd'].rank(axis=0,method='first')/len(temp_merged_df))*100
    temp_merged_df = temp_merged_df[col2]


    # 初始化并更新数据库
    updater = MySQLUpdater(db_params, 'daily_price', temp_merged_df, '證券代號','2')
    updater.update_records()
updater.close_connection()



# df_rs_week = pd.concat([df_rs_week,df_rs70])
# df_rsall_week = pd.concat([df_rsall_week,df_rs])
# # 順便統計細產業佔比
# rs70_sub_category = get_sub_category_list(sub_category_list,df_rs70,day)                        
# df_rs70_category_week = pd.concat([df_rs70_category_week,rs70_sub_category])


C:\Users\USER\AppData\Local\Temp\ipykernel_31508\1971781208.py:16: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df = merged_df.fillna(0)


In [233]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime, timedelta

# 設置MySQL資料庫連接
db_user = 'root'
db_password = '19970730'
db_host = '127.0.0.1'
db_name = 'sql_stock'

# 創建資料庫連接引擎
engine = create_engine(f'mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}')

# 設置日期範圍
#today = datetime(2024, 5, 25)
today = datetime.now().date()
one_year_ago = today - timedelta(days=365)

# 格式化日期
today_str = today.strftime('%Y%m%d')
one_year_ago_str = one_year_ago.strftime('%Y%m%d')



# SQL 查詢 個股
query_stock = f"""
SELECT *
FROM daily_price
WHERE date >= '{one_year_ago_str}' AND date <= '{today_str}'
"""

# 使用 pd.read_sql 來執行查詢並讀取數據到 DataFrame
stock_df_raw = pd.read_sql(query_stock, engine)



In [226]:


# 定义函数来提取数值
def extract_values(row):
    indicator = row.iloc[0]
    values = []
    
    if indicator == '上漲(漲停)':
        values.append(('上漲家數', int(row.iloc[2].split('(')[0].replace(',', ''))))
        values.append(('漲停家數', int(row.iloc[2].split('(')[1].strip(')'))))
    elif indicator == '下跌(跌停)':
        values.append(('下跌家數', int(row.iloc[2].split('(')[0].replace(',', ''))))
        values.append(('跌停家數', int(row.iloc[2].split('(')[1].strip(')'))))
    elif indicator == '持平':
        values.append(('持平家數', int(row.iloc[2].replace(',', ''))))
    elif indicator == '未成交':
        values.append(('未成交家數', int(row.iloc[2].replace(',', ''))))
    elif indicator == '無比價':
        values.append(('無比價家數', int(row.iloc[2].replace(',', ''))))

    return values

df_twse_ud = pd.DataFrame(jsondata['data8'])

# 提取所有数值
extracted_data = []
for idx, row in df_twse_ud.iterrows():
    
    extracted_data.extend(extract_values(row))

extracted_data

df_twse_updown = pd.DataFrame(extracted_data, columns=['指標', '數值'])

# 轉置 
df_twse_updown = df_twse_updown.T
df_twse_updown.columns = df_twse_updown.iloc[0]
df_twse_updown = df_twse_updown[1:]
df_twse_updown.drop(['未成交家數', '無比價家數'], axis=1, inplace=True)
df_twse_updown.rename(columns={'上漲家數': '上市上漲家數','漲停家數': '上市漲停家數','下跌家數': '上市下跌家數','跌停家數': '上市跌停家數','持平家數': '上市持平家數'}, inplace=True)
df_twse_updown['Date'] = Date
df_twse_updown

指標,上市上漲家數,上市漲停家數,上市下跌家數,上市跌停家數,上市持平家數,Date
數值,9,0,997,428,2,20240805


In [197]:
import requests
import pandas as pd
from io import StringIO
import re

url = "https://www.tpex.org.tw/web/stock/aftertrading/market_highlight/highlight_result.php?l=zh-tw&o=csv&d="
date = '113/08/05'
# 下載股價
r = requests.get(url + date)

data = []
for line in StringIO(r.text):
    try:
        data.append(line.split(','))
    except Exception as e:
        print(f"Error processing line: {line}, error: {e}")

df = pd.DataFrame(data)
extracted_data = []

for i in range(len(df)):
    df_col = df.iloc[i]
    # 清理数据，移除 \r\n 和多余的空格
    cleaned_data = df_col.dropna().str.replace('\r\n', '').str.strip()

    # 提取数值并创建 DataFrame

    for item in cleaned_data:
        match1 = re.match(r'"(上漲家數|漲停家數|下跌家數|跌停家數|平盤家數):\s*(\d+)"', item)
        if match1:
            indicator = match1.group(1)
            value = int(match1.group(2))
            extracted_data.append((indicator, value))

    df_tpex_updown = pd.DataFrame(extracted_data, columns=['指標', '數值'])
df_tpex_updown

# 轉置 
df_tpex_updown = df_tpex_updown.T
df_tpex_updown.columns = df_tpex_updown.iloc[0]
df_tpex_updown = df_tpex_updown[1:]
df_tpex_updown.rename(columns={'上漲家數': '上櫃上漲家數','漲停家數': '上櫃漲停家數','下跌家數': '上櫃下跌家數','跌停家數': '上櫃跌停家數','平盤家數': '上櫃持平家數'}, inplace=True)
df_tpex_updown['Date'] = Date
df_tpex_updown



指標,上櫃上漲家數,上櫃漲停家數,上櫃下跌家數,上櫃跌停家數,上櫃持平家數,Date
數值,12,1,806,371,3,20240805


In [210]:
# 保留索引水平合并
df_combined = pd.merge(df_tpex_updown, df_twse_updown, on='Date')
df_combined

df_combined['總上漲家數'] = df_tpex_updown['上櫃上漲家數'].iloc[0] + df_twse_updown['上市上漲家數'].iloc[0] 
df_combined['總下跌家數'] = df_tpex_updown['上櫃下跌家數'].iloc[0] + df_twse_updown['上市下跌家數'].iloc[0] 
df_combined['總漲停家數'] = df_tpex_updown['上櫃漲停家數'].iloc[0] + df_twse_updown['上市漲停家數'].iloc[0] 
df_combined['總跌停家數'] = df_tpex_updown['上櫃跌停家數'].iloc[0] + df_twse_updown['上市跌停家數'].iloc[0] 
df_combined['總持平家數'] = df_tpex_updown['上櫃持平家數'].iloc[0] + df_twse_updown['上市持平家數'].iloc[0] 

# 做多 1.35 ~ 0.6 做空
df_combined['上櫃上漲下跌比']  = round(df_tpex_updown['上櫃上漲家數'].iloc[0] / df_tpex_updown['上櫃下跌家數'].iloc[0], 2 )
df_combined['上市上漲下跌比']  = round(df_twse_updown['上市上漲家數'].iloc[0] / df_twse_updown['上市下跌家數'].iloc[0], 2 )
df_combined['總上漲下跌比']    = round(df_combined['總上漲家數'].iloc[0]  / df_combined['總下跌家數'].iloc[0], 2 )

df_combined

指標,上櫃上漲家數,上櫃漲停家數,上櫃下跌家數,上櫃跌停家數,上櫃持平家數,Date,上市上漲家數,上市漲停家數,上市下跌家數,上市跌停家數,上市持平家數,總上漲家數,總下跌家數,總漲停家數,總跌停家數,總持平家數,上櫃上漲下跌比,上市上漲下跌比,總上漲下跌比
0,12,1,806,371,3,20240805,9,0,997,428,2,21,1803,1,799,5,0.01,0.01,0.01
